In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from collections import Counter
import random
# import matplotlib.pyplot as plt
import numpy as np
# from sklearn.model_selection import train_test_split
# from tqdm.notebook import tqdm

In [2]:
# Número de GPUs disponibles. Usar 0 para modo CPU.
ngpu = 1

# Fuente de la cual queremos obtener los datos
source = 'corenlp'

In [3]:
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

In [4]:
ix_to_cat = ['C1', 'C2', 'E1', 'E2', 'E3', 'E4','V1']
cat_to_ix = { cat: ix for ix, cat in enumerate(ix_to_cat) }

num_cat = len(ix_to_cat) # Tamaño de salida de la red

cat_to_ix

{'C1': 0, 'C2': 1, 'E1': 2, 'E2': 3, 'E3': 4, 'E4': 5, 'V1': 6}

In [5]:
ix_to_w = []
with open(f'./../Embeddings/word_labels_{source}.txt', 'r', encoding='utf-8') as f:
    ix_to_w = f.read().splitlines()
    
w_to_ix = { w: ix for ix, w in enumerate(ix_to_w) }

tam_vocab = len(ix_to_w)

ix_to_w[:5]

[',', 'de', 'que', 'la', 'a']

In [6]:
sentences = []
with open(f'./Corpus/Test/test-batch250-300.txt', 'r', encoding='utf-8') as f:
    sentences = [s.split() for s in f.read().splitlines()]

[' '.join(s) for s in sentences[:5]]

['La única forma de lograr un cambio profundo en México es buscando soluciones diferentes a los problemas , dijo .',
 'También estuvo presente Celina Saavedra , ex diputada de la Asamblea Legislativa del Distrito Federal ( ALDF ) y ex coordinadora de la esntructura en la CDMX de Margarita Zavala .',
 'En el gobierno de coalición , el Secretario de Gobernación fungirá como Jefe de Gabinete y pertenecerá a una fuerza política distinta a la del Presidente de la República .',
 'Los jóvenes son el motor de México y tendrán todo el apoyo : José Antonio Meade .',
 "También advierte del ' charlatán ' que ofrece amnistía y perdón a delincuentes ."]

In [7]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = w_to_ix[BOS]
ixEOS = w_to_ix[EOS]
ixUNK = w_to_ix[UNK]

In [8]:
def unk_w_to_ix(w):
    try:
        return w_to_ix[w]
    except KeyError:
        return ixUNK

In [9]:
X = [
    # Convierto las palabras (en minúsculas) a índices
    [ unk_w_to_ix(w.lower()) for w in sent ] 
    for sent in sentences
]

[str(x) for x in X[:5]]

['[3, 1158, 407, 1, 302, 19, 115, 819, 8, 20, 18, 2584, 1544, 1331, 4, 9, 397, 0, 35, 6]',
 '[101, 339, 1474, 7473, 7473, 0, 345, 3562, 1, 3, 1952, 7473, 15, 3627, 214, 202, 7473, 164, 7, 345, 1404, 1, 3, 7473, 8, 3, 7473, 1, 1240, 2012, 6]',
 '[8, 5, 31, 1, 38, 0, 5, 638, 1, 1352, 5589, 28, 761, 1, 1771, 7, 7473, 4, 24, 674, 161, 4464, 4, 3, 15, 53, 1, 3, 71, 6]',
 '[9, 112, 57, 5, 1806, 1, 20, 7, 439, 66, 5, 143, 40, 48, 50, 34, 6]',
 '[101, 3328, 15, 887, 5758, 887, 2, 1044, 841, 7, 1449, 4, 585, 6]']

In [10]:
X = [np.asarray(x, dtype=np.long) for x in X]

In [11]:
def np_to_long_tensor(list_of_lists):
    return [
        torch.from_numpy(l).long().to(device)
        for l in list_of_lists
    ]

In [12]:
X_test = np_to_long_tensor(X)
N_test = len(X_test)
N_test

50

In [13]:
class PersuasionLSTM(nn.Module):
    def __init__(self, D_in, D_emb, D_lstm, D_out):
        super(PersuasionLSTM, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        # self.dropout = nn.Dropout(p=0.2)
        self.lstm = nn.LSTM(input_size=D_emb, hidden_size=D_lstm)
        self.linear = nn.Linear(in_features=D_lstm, out_features=D_out)
        self.sig = nn.Sigmoid()
        

    def forward(self, sentence):
        T = len(sentence)
        embeddings = self.embedding(sentence).view(T, 1, -1)
        # embeddings = self.dropout(embeddings)
        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)
        preact_out = self.linear(lstm_out).view(T, -1)
        return self.sig(preact_out[-1])

In [14]:
# Dimensiones de la red neuronal
D_in = tam_vocab
D_emb = 64
D_lstm = 32
D_out = num_cat

In [15]:
persuasion = PersuasionLSTM(D_in, D_emb, D_lstm, D_out).to(device)
persuasion.eval()

PersuasionLSTM(
  (embedding): Embedding(7474, 64)
  (lstm): LSTM(64, 32)
  (linear): Linear(in_features=32, out_features=7, bias=True)
  (sig): Sigmoid()
)

In [18]:
dir_path = './modelsaves/'
filename = f'model-corenlp-emb_64-lstm_32-seed_29-epochs_100-last-trained'
persuasion.load_state_dict(torch.load(dir_path+filename))

<All keys matched successfully>

In [19]:
def print_ix_sentence(sentence):
    print(' '.join(ix_to_w[ix] for ix in sentence.data))

def print_ix_cat(probs):
    categorias = []
    print('Categorías inferidas: ', end='')
    for i, p_i in enumerate(probs.data):
        if p_i.item() > 0.55:
            categorias.append(ix_to_cat[i])
    if len(categorias) == 0:
        print('NA')
    else:
        print(', '.join(categorias))
#     print('Probabilidades')
#     print('\t'.join(ix_to_cat))
#     print('\t'.join([f'{p_i.item():0.3f}' for p_i in probs.data]))

1. **Construcción del emisor/candidato (C1):** el candidato se coloca como referente principal del acto discursivo para exaltar sus cualidades personales y erigirse, a los ojos de sus interlocutores, como un líder en quien es adecuado confiar para el desarrollo de acciones de gobierno a favor de la sociedad; esto lo lleva a cabo mediante realizaciones léxicas de la primera persona, en singular o plural, y formas flexivas de verbos y posesivos para referirse a sí mismo, ya sea como individuo o como miembro de una colectividad. Para el caso de la campaña presidencial que nos ocupa, las cualidades del candidato emitidas por un tercero en el marco de una tarea proselitista, quedarán incluidas en esta categoría, ya que contribuyen a construir la imagen positiva del contendiente.

2. **Promesa de campaña (C2):** a través de piezas discursivas, orales o escritas, el candidato oferta, destaca o compromete acciones que proyecta convertir en políticas públicas a favor de la población, si obtiene el triunfo en las urnas, con lo que implícitamente solicita el voto ciudadano que lo llevaría al poder.

3. **Construcción del adversario (E1):** el mensaje es dirigido a su contraparte en la contienda electoral y está construido a manera de réplica y puede usar diferentes figuras retóricas (metáfora, ironía, metonimia) para conformar en el receptor una imagen determinada del adversario, que esencialmente tiene finalidad de descrédito.

4. **Exageración de la información (E2):** se destacan los datos favorables a los fines persuasivos, con el objetivo de crear una idea positiva en la mente del receptor referente al tema del que se habla, o incluso negativa respecto de información relativa al adversario; el persuasor desfigura el sentido original del acontecimiento, nos dice Roiz (1994), mediante códigos diferentes: humorístico, burlesco, cínico, entre otros.

5. **Recurso retórico (E3):** son variables basadas en figuras retóricas (metáfora, ironía, hipérbole, personificación, pleonasmo, perífrasis) que pueden fortalecer el efecto persuasivo, de acuerdo con Reardon (1991).

6. **Apelación al miedo (E4):** el candidato emite mensajes dentro de su alocución que pretenden provocar sentimientos de aprensión, desasosiego o preocupación con respecto al adversario o a sus propuestas. 

7. **Llamado al voto (V1):** el emisor del mensaje hace una invitación directa al ciudadano para que apoye su proyecto político el día de la elección al depositar el sufragio a su favor.


In [20]:
with torch.no_grad():
    for i, x in enumerate(X_test):
        print('='*50)
        print(' '.join(sentences[i]))
#         print_ix_sentence(x)
        pred = persuasion(x)
        print()
        print_ix_cat(pred)
        print('='*50)
        print()

La única forma de lograr un cambio profundo en México es buscando soluciones diferentes a los problemas , dijo .

Categorías inferidas: C2

También estuvo presente Celina Saavedra , ex diputada de la Asamblea Legislativa del Distrito Federal ( ALDF ) y ex coordinadora de la esntructura en la CDMX de Margarita Zavala .

Categorías inferidas: C2

En el gobierno de coalición , el Secretario de Gobernación fungirá como Jefe de Gabinete y pertenecerá a una fuerza política distinta a la del Presidente de la República .

Categorías inferidas: NA

Los jóvenes son el motor de México y tendrán todo el apoyo : José Antonio Meade .

Categorías inferidas: NA

También advierte del ' charlatán ' que ofrece amnistía y perdón a delincuentes .

Categorías inferidas: NA

" Nosotros sí la vamos a cambiar , en dos grandes ejes : Por un lado la prevención , atendiendo las causas más profundas y , por otro lado , vamos a enfrentar el problema , pero con una nueva estrategia " .

Categorías inferidas: C1, E3
